In [2]:
from torch.utils.data import Dataset
from fl_puf.Utils.tabular_data_loader import get_tabular_numpy_dataset
import torch.nn as nn
import numpy as np
import torch 
from torch import nn

/home/luca.corbucci/fl_puf/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-21 17:27:19,674	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
class TabularDataset(Dataset):
    def __init__(self, x, z, y):
        """
        Initialize the custom dataset with x (features), z (sensitive values), and y (targets).

        Args:
        x (list of tensors): List of input feature tensors.
        z (list): List of sensitive values.
        y (list): List of target values.
        """
        self.samples = x
        self.sensitive_features = z
        self.sensitive_attribute = z
        self.gender = z
        self.classes = y
        self.targets = y

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Get a single data point from the dataset.

        Args:
        idx (int): Index to retrieve the data point.

        Returns:
        sample (dict): A dictionary containing 'x', 'z', and 'y'.
        """
        x_sample = self.samples[idx]
        z_sample = self.sensitive_features[idx]
        y_sample = self.targets[idx]

        return x_sample, z_sample, y_sample


class LinearClassificationNet(nn.Module):
    """
    A fully-connected single-layer linear NN for classification.
    """

    def __init__(self, input_size=11, output_size=2):
        super(LinearClassificationNet, self).__init__()
        self.layer1 = nn.Linear(input_size, output_size, bias=False)

    def forward(self, x):
        x = self.layer1(x.float())
        return x


In [4]:
X, z, y = get_tabular_numpy_dataset(
        dataset_name="adult",
        num_sensitive_features=1,
        dataset_path="./data/Tabular/adult/",
    )
print(X.shape)

Using ['sex_binary'] as sensitive feature(s).
(32561, 110)


In [4]:
custom_dataset = TabularDataset(
        x=X, z=z, y=y
)

train_loader = torch.utils.data.DataLoader(
    custom_dataset,
    batch_size=32,
)

In [23]:
def accuracy(preds, labels):
    return (preds == labels).mean()

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    for i, (images, sensitive_value,  target) in enumerate(train_loader):
        optimizer.zero_grad()
        images = images.to(device)
        target = target.long()
        target = target.to(device)

        

        # compute output
        output = model(images)
        
        loss = criterion(output, target)

        

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()

        # measure accuracy and record loss
        acc = accuracy(preds, labels)

        losses.append(loss.item())
        top1_acc.append(acc)

        loss.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
            )

In [6]:
model = LinearClassificationNet(input_size=110, output_size=2)

In [24]:
train(model=model, 
        train_loader=train_loader, 
        optimizer=torch.optim.Adam(
                model.parameters(),
                lr=0.01,
            ), 
        epoch=2, 
        device="cpu")

	Train Epoch: 2 	Loss: 0.335527 Acc@1: 84.218750 
	Train Epoch: 2 	Loss: 0.333461 Acc@1: 84.468750 
	Train Epoch: 2 	Loss: 0.331134 Acc@1: 84.552083 
	Train Epoch: 2 	Loss: 0.327464 Acc@1: 84.730469 
	Train Epoch: 2 	Loss: 0.326217 Acc@1: 84.828125 
